<a href="https://colab.research.google.com/github/Lasrixx/SMSTextClassifier/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Put files into dataframes
train_df = pd.read_table(train_file_path, header=0, names=['label','text'])
print(train_df)
test_df = pd.read_table(test_file_path, header=0, names=['label','text'])
#print(test_df)

In [ ]:
train_df.shape

In [ ]:
# Convert ham and spam to boolean - we have been told ham is 0 and spam is 1
train_df['label'] = train_df['label'].replace("ham", 0)
train_df['label'] = train_df['label'].replace("spam", 1)
test_df['label'] = test_df['label'].replace("ham", 0)
test_df['label'] = test_df['label'].replace("spam", 1)

In [ ]:
print(train_df)

In [ ]:
# Separate data and labels
train_labels = train_df.pop('label')
test_labels = test_df.pop('label')

In [ ]:
print(train_df)
print(train_labels)

In [ ]:
# Want to partition part of the train dataset for validation purposes
# Split the dataset into 80% train, 20% validation
val_split = 0.2
split_index = int(len(train_df) * val_split)

print(split_index)

validation_data = train_df.to_numpy()[:split_index]
train_data = train_df.to_numpy()[split_index:]
validation_labels = train_labels.to_numpy()[:split_index]
train_labels = train_labels.to_numpy()[split_index:]

In [ ]:
# Will use a pre-trained NLP model as a base then build on
# Using tensorflow hub's google/nnlm-en-dim50-with-normalization/2 
model = "https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
# Build on the model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
# Compile the model
# BinaryCrossEntropy is useful for predicting probabilities 
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
# Train the model
history = model.fit(train_data,
                    train_labels,
                    epochs=40,
                    batch_size=512,
                    validation_data=(validation_data, validation_labels),
                    verbose=1)

In [ ]:
model.evaluate(test_df,test_labels)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict([pred_text])[0]
  output = "ham"
  if pred[0] > 0.5:
    output = "spam"
  
  prediction = [pred[0],output]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
